In [14]:
import pandas as pd
import numpy as np
import re
import json

In [22]:
dataFile = './speciesdata.csv'
df = pd.read_csv(dataFile)
df.head()

,family,scientificName,vernacularNames,Endêmico (Chapada),Endêmico (Cerrado),...,Atividade (noturno/diurno),Tamanho medio (mm) - fêmea,Tamanho medio (mm) - macho,Meses de Ocorrência (checar planilha instagram),Chances de encontros (muito raro/raro/frequente/muito frequente)
0,AROMOBATIDAE (sapos-foguete),"Allobates goianus (Bokermann, 1975)",Sapo-foguete-de-Goiás,não,sim,...,diurno,17.0,17.0,out-mar,Muito raro
1,BUFONIDAE (sapos-cururu),"Rhaebo guttatus (Schneider, 1799)",Cururu-de-couro,não,não,...,noturno,174.0,121.0,NaN,Raro
2,BUFONIDAE (sapos-cururu),"Rhinella mirandaribeiroi (Gallardo, 1965)",Sapo Cururu,não,sim,...,noturno,61.0,55.0,out-fev,Raro
3,BUFONIDAE (sapos-cururu),"Rhinella ocellata (Günther, 1858)",Sapo Cururu,não,não,...,noturno,44.0,44.0,out-mar,Frequente
4,BUFONIDAE (sapos-cururu),"Rhinella rubescens (Lutz, 1925)",Cururu-vermelho,não,sim,...,noturno,90.0,90.0,jan-dez,Frequente


In [16]:
df.columns

Index(['family', 'scientificName', 'vernacularNames', 'Endêmico (Chapada)',
       'Endêmico (Cerrado)', 'Red List',
       'Postura dos ovos (habitat breeding)*',
       'Poleiro (tipical calling perch)* ', 'Atividade (noturno/diurno)',
       'Tamanho medio (mm) - fêmea', 'Tamanho medio (mm) - macho',
       'Meses de Ocorrência (checar planilha instagram)',
       'Chances de encontros (muito raro/raro/frequente/muito frequente)'],
      dtype='object')

In [23]:
df.rename( {'scientificName':'speciesName',
            'Postura dos ovos (habitat breeding)*': 'habitat_breeding',
            'Chances de encontros (muito raro/raro/frequente/muito frequente)': 'detectability'}, axis=1, inplace=True)

In [27]:
df.head()

,family,speciesName,vernacularNames,Endêmico (Chapada),Endêmico (Cerrado),...,Tamanho medio (mm) - macho,Meses de Ocorrência (checar planilha instagram),detectability,scientificName,scientificNameAuthorship
0,AROMOBATIDAE (sapos-foguete),"Allobates goianus (Bokermann, 1975)",Sapo-foguete-de-Goiás,não,sim,...,17.0,out-mar,Muito raro,Allobates goianus,"Bokermann, 1975"
1,BUFONIDAE (sapos-cururu),"Rhaebo guttatus (Schneider, 1799)",Cururu-de-couro,não,não,...,121.0,NaN,Raro,Rhaebo guttatus,"Schneider, 1799"
2,BUFONIDAE (sapos-cururu),"Rhinella mirandaribeiroi (Gallardo, 1965)",Sapo Cururu,não,sim,...,55.0,out-fev,Raro,Rhinella mirandaribeiroi,"Gallardo, 1965"
3,BUFONIDAE (sapos-cururu),"Rhinella ocellata (Günther, 1858)",Sapo Cururu,não,não,...,44.0,out-mar,Frequente,Rhinella ocellata,"Günther, 1858"
4,BUFONIDAE (sapos-cururu),"Rhinella rubescens (Lutz, 1925)",Cururu-vermelho,não,sim,...,90.0,jan-dez,Frequente,Rhinella rubescens,"Lutz, 1925"


### Species name, id and Authorship

In [25]:
df['scientificName'] = df['speciesName'].apply(lambda x: re.findall( '^\w+(?: cf.| aff.)? \w+\.?', str(x))[0])

In [26]:
df['scientificNameAuthorship'] = df[['speciesName','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)
df['scientificNameAuthorship'] = df['scientificNameAuthorship'].apply(lambda x: re.findall( '[\w,\s\-&]+', str(x) )).apply(lambda x: ''.join(x).strip())

/tmp/ipykernel_441642/1646656252.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['scientificNameAuthorship'] = df[['speciesName','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)


In [28]:
ids = df['scientificName'].str.replace('\s','-').str.replace('[.]','').str.lower()
df.insert(0, 'id', ids)

In [29]:
df.drop('speciesName', axis=1, inplace=True)

### Family name and id

In [30]:
df['family'] = df['family'].apply(lambda x: x.capitalize())
df['family_id'] = df['family'].str.lower()

### Vernacular name

In [31]:
def extractVernacularNames(string):
    if pd.isna(string): string=''
    names = re.split('/|;|,',string)
    names = [ n.strip().replace(' ','-').capitalize() for n in names ]
    if len(names)==1 and names[0]=='':
        return []
    else:
        return names

In [32]:
df['vernacularNames'] = df['vernacularNames'].apply(extractVernacularNames)

### Red list

In [33]:
df['Red List'] = df['Red List'].str.lower()
df.loc[ df['Red List']=='não avaliada','Red List']='ne'

In [34]:
df = pd.get_dummies(data=df, prefix='redlist', columns=['Red List'])

### Endemicidade

In [35]:
df['endemic_cerrado'] = df['Endêmico (Cerrado)'].apply(lambda x: 1 if x=='sim' else 0)
df['endemic_chapada'] = df['Endêmico (Chapada)'].apply(lambda x: 1 if x=='sim' else 0)
df.drop('Endêmico (Chapada)', axis=1, inplace=True)
df.drop('Endêmico (Cerrado)', axis=1, inplace=True)

In [36]:
# Helpers

getInsideParentheses = lambda x: [ str.lower(e) for e in re.findall( '\((.{0,5})\)', str(x) ) ]

### Detectability

In [44]:
df['detectability'] = df['detectability']\
    .str.replace('[^\w]','',regex=True)\
    .str.replace('ê','e')\
    .str.lower()

In [46]:
subst_dict = {
    'muitofrequente': 'ff',
    'frequente':'f',
    'raro': 'r',
    'muitoraro': 'rr'
}

df['detectability'] = df['detectability'].apply(lambda x: subst_dict[x] if x is not np.NaN else x)
df = pd.get_dummies(df, prefix='detectability',columns=['detectability'])

### Poleiro

In [47]:
df['poleiro'] = df['Poleiro (tipical calling perch)* '].apply( getInsideParentheses )
df.drop('Poleiro (tipical calling perch)* ', axis=1, inplace=True)

In [48]:
poleiro_types = list(set( el for ls in df['poleiro'] for el in ls))

In [49]:
for tp in poleiro_types:
    df[f'tcp_{tp}'] = df['poleiro'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [50]:
df.drop('poleiro', axis=1, inplace=True)

### Habitat breeding

In [51]:
matches = df['habitat_breeding'].str.lower().str.extractall('\((\w+)\)')
hb_dummies = pd.get_dummies(matches, prefix="habitat_breeding").groupby(level=0).sum()

In [52]:
hb_dummies = hb_dummies.reindex(df.index).fillna(0).astype(int)

In [53]:
df = pd.concat([df,hb_dummies], axis=1)

In [54]:
df.drop('habitat_breeding',axis=1, inplace=True)

### Phytophysiognomies categoricals

In [55]:
df.rename({'Categorias fitofisionomias (guia)': 'phytos'}, axis=1, inplace=True)

In [57]:
df

,id,family,vernacularNames,Atividade (noturno/diurno),Tamanho medio (mm) - fêmea,...,tcp_lb,tcp_hb,habitat_breeding_le,habitat_breeding_lo,habitat_breeding_tr
0,allobates goianus,Aromobatidae (sapos-foguete),[Sapo-foguete-de-goiás],diurno,17.0,...,0,0,0,0,1
1,rhaebo guttatus,Bufonidae (sapos-cururu),[Cururu-de-couro],noturno,174.0,...,0,0,1,0,0
2,rhinella mirandaribeiroi,Bufonidae (sapos-cururu),[Sapo-cururu],noturno,61.0,...,0,0,1,0,0
3,rhinella ocellata,Bufonidae (sapos-cururu),[Sapo-cururu],noturno,44.0,...,0,0,0,0,1
4,rhinella rubescens,Bufonidae (sapos-cururu),[Cururu-vermelho],noturno,90.0,...,0,0,1,1,0
5,rhinella diptycha,Bufonidae (sapos-cururu),[Sapo-cururu],noturno,175.0,...,0,0,1,1,0
6,barycholos ternetzi,Strabomantidae (rãs-de-chuva),[Rãzinha-da-chuva],diurno,26.5,...,0,0,0,0,1
7,ameerega flavopicta,Dendrobatidae (rãs-venenosos),[Sapo-flecha],diurno,22.0,...,0,0,0,0,1
8,aplastodiscus lutzorum,Hylidae (pererecas-verdadeiras),[Perereca-verde-da-mata],noturno,33.7,...,1,1,0,1,0
9,bokermannohyla pseudopseudis,Hylidae (pererecas-verdadeiras),[Perereca-de-pedra],noturno,61.0,...,1,0,0,1,0


In [56]:
replaces = {
} 
df['phytos'] = df['phytos'].str.lower().str.strip().str.split('\s*,\s*')\
    .apply(lambda x: [replaces.get(i,i) for i in x] if isinstance(x,list) else '')\
    .str.join(',')

KeyError: 'phytos'

In [ ]:
df['phytos']

0                      fg
1                   fg,ce
2                  css,aa
3               fc,css,aa
4     fc,fg,ce,css,c,v,aa
5     fc,fg,ce,css,c,v,aa
6            fc,fg,ce,css
7                 scr,ccr
8                   fc,fg
9                cr,fg,fc
10                  c,css
11               c,css,aa
12                       
13               c,css,aa
14                       
15               c,css,aa
16              fc,fg,css
17                  fc,fg
18                fc,fg,c
19                  fc,fg
20             c,fc,fg,aa
21                       
22         c,css,fc,fg,aa
23                  c,css
24                       
25                       
26                  c,css
27               c,css,aa
28         c,css,fc,fg,aa
29                      c
             ...         
33               c,css,cr
34               fc,fg,aa
35                  c,css
36                  c,css
37                  c,css
38                      c
39       c,css,aa,fc,fg,v
40       c,c

### Habitat

In [58]:
df['habitat'] = df['Habitat de vida (mata ou floresta/area aberta/pedras)*'].apply( getInsideParentheses)
df.drop('Habitat de vida (mata ou floresta/area aberta/pedras)*', axis=1, inplace=True)

KeyError: 'Habitat de vida (mata ou floresta/area aberta/pedras)*'

In [ ]:
habitat_types = list(set( el for ls in df['habitat'] for el in ls ))

In [ ]:
for tp in habitat_types:
    df[f'habitat_{tp}'] = df['habitat'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [ ]:
df.drop('habitat', axis=1, inplace=True)

### Atividade

In [59]:
df['Atividade (noturno/diurno)'].value_counts()

Atividade (noturno/diurno)
noturno              48
diurno                3
noturno e diurno      2
noturno e diurno      1
Name: count, dtype: int64

In [60]:
df['atividade_diu'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'diurno' in str(x) else 0)
df['atividade_not'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'noturno' in str(x) else 0)
df.drop('Atividade (noturno/diurno)', axis=1, inplace=True)

### Tamanho

In [67]:
df['tamanho_femea'] = df['Tamanho medio (mm) - fêmea'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))
df['tamanho_macho'] = df['Tamanho medio (mm) - macho'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))

In [68]:
df['tamanho_femea_med'] = df['tamanho_femea'].apply( lambda l: sum([float(i) for i in l])/len(l) if len(l)>0 else None )
df['tamanho_macho_med'] = df['tamanho_macho'].apply( lambda l: sum([float(i) for i in l])/len(l) if len(l)>0 else None )

In [ ]:
#df['tamanho_femea_min'] = df['tamanho_femea'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_femea_max'] = df['tamanho_femea'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_macho_min'] = df['tamanho_macho'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_macho_max'] = df['tamanho_macho'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)

#df['tamanho_macho_max'].replace(np.NaN, df['tamanho_femea_max'], inplace=True)
#df['tamanho_macho_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)
#df['tamanho_femea_max'].replace(np.NaN, df['tamanho_macho_max'], inplace=True)
#df['tamanho_femea_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)

In [69]:
df.drop('tamanho_femea', axis=1, inplace=True)
df.drop('tamanho_macho', axis=1, inplace=True)
df.drop('Tamanho medio (mm) - fêmea', axis=1, inplace=True)
df.drop('Tamanho medio (mm) - macho', axis=1, inplace=True)

In [70]:
df['tamanho_macho_med'].replace(np.NaN, df['tamanho_femea_med'], inplace=True)
df['tamanho_femea_med'].replace(np.NaN, df['tamanho_macho_med'], inplace=True)
df['tamanho_med'] = df[['tamanho_macho_med','tamanho_femea_med']].mean(axis=1)

ValueError: Series.replace cannot use dict-value and non-None to_replace

### Meses de ocorrência

1. Extract month ranges from strings

In [ ]:
import unicodedata

normalize_str = lambda x: unicodedata.normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8').lower().strip()
months=['janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro',
        'jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']

In [ ]:
df['Meses de Ocorrência'] = df['Meses de Ocorrência'].apply(lambda x: normalize_str(x) if pd.notnull(x) else x)\
    .replace('chuva','out-mar')\
    .replace('ano todo', 'jan-dez')\
    .replace('seca-chuva','jan-dez')\
    .replace('chuva ate marco','out-mar')\
    .str.findall("("+"|".join(months)+")")\
    .apply(lambda x: [ i[:3] for i in x] if isinstance(x,list) else x)\
    .str.join(sep='-')\

In [ ]:
def occurrence_months(m_start, m_end):
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    idx_mstart = d.index(m_start)
    idx_mend = d.index(m_end)
    
    if idx_mstart > idx_mend:
        return d[idx_mstart:]+d[:idx_mend+1]
    else:
        return d[idx_mstart:idx_mend+1]
    
    
def months_to_array(mths):
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    return [ 1 if i in mths else 0 for i in d ]

In [ ]:
df['month_vec'] = df['Meses de Ocorrência'].str.split('-').apply(
    lambda x: months_to_array(occurrence_months(*x)) if isinstance(x,list) else [0 for i in range(12)]
)

### Temporarily remove unused fields

In [ ]:
df.drop('Distribuição - SITE REUBER (espécies do DF)', axis=1, inplace=True)
df.drop('Ameaças - SITE REUBER (espécies do DF)', axis=1, inplace=True)

### Remove unwanted species

In [ ]:
spp_to_remove = ['boana-sp','leptodactylus-aff-cunicularius','elachistocleis-sp']

In [ ]:
df = df[ df['id'].apply(lambda x: x not in spp_to_remove) ]

### Sort dataframe by Family and id

In [ ]:
df = df.sort_values(by=['family','id'])

### Write to json

In [ ]:
species_data_path = '../_data/species.json'

d = df.to_dict(orient='records')

with open(species_data_path, 'w') as f:
    json.dump(d, f,indent=1, ensure_ascii=False)